In [2]:
import pandas as pd
import numpy as np
import json
import seaborn as sns
import matplotlib.pyplot as plt
from colorama import Fore, Style, init;
import xgboost as xgb
import lightgbm as lgb
import torch

In [3]:
def display_df(df, name):
    '''Display df shape and first row '''
    PrintColor(text = f'{name} data has {df.shape[0]} rows and {df.shape[1]} columns. \n ===> First row:')
    display(df.head(1))

# Color printing    
def PrintColor(text:str, color = Fore.BLUE, style = Style.BRIGHT):
    '''Prints color outputs using colorama of a text string'''
    print(style + color + text + Style.RESET_ALL); 

In [6]:
CSV_DIR = "predict-energy-behavior-of-prosumers/"

train = pd.read_csv(CSV_DIR + "train.csv")
client = pd.read_csv(CSV_DIR + "client.csv")
historical_weather = pd.read_csv(CSV_DIR + "historical_weather.csv")
forecast_weather = pd.read_csv(CSV_DIR + "forecast_weather.csv")
electricity = pd.read_csv(CSV_DIR + "electricity_prices.csv")
gas = pd.read_csv(CSV_DIR + "gas_prices.csv")

In [7]:
revealed_targets_columns = ['county', 'is_business', 'product_type', 'target', 'is_consumption', 'datetime', 'prediction_unit_id']
revealed_targets = train[revealed_targets_columns].copy()

In [8]:
display_df(train, 'train')
display_df(revealed_targets, 'revealed_targets')
display_df(client, 'client')
display_df(historical_weather, 'historic weather')
display_df(forecast_weather, 'forecast weather')
display_df(electricity, 'electricity prices')
display_df(gas, 'gas prices')

train data has 2018352 rows and 9 columns. 
 ===> First row:


,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0


revealed_targets data has 2018352 rows and 7 columns. 
 ===> First row:


,county,is_business,product_type,target,is_consumption,datetime,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0


client data has 41919 rows and 7 columns. 
 ===> First row:


...

,latitude,longitude,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,data_block_id,forecast_datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
0,57.6,21.7,2021-09-01 00:00:00+00:00,1,15.655786,11.553613,0.904816,0.019714,0.0,0.905899,-0.411328,-9.106137,1,2021-09-01 01:00:00+00:00,0.0,0.0,0.0,0.0


electricity prices data has 15286 rows and 4 columns. 
 ===> First row:


,forecast_date,euros_per_mwh,origin_date,data_block_id
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1


gas prices data has 637 rows and 5 columns. 
 ===> First row:


,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id
0,2021-09-01,45.23,46.32,2021-08-31,1
